In [12]:
import pandas as pd
import numpy as np

mturk = pd.read_csv("Study4_filtered_with_0 (1).csv", low_memory=False).iloc[2:]
mturk = mturk.loc[mturk["stop_disp"]==mturk["stop_disp"]]
mturk = mturk[mturk["Duration (in seconds)"].apply(int)>900]

a = len(mturk)




k = lambda x: tuple(x.split(":")[0].replace(".png","").split("_"))
v = lambda x: float(x.split(":")[1] if x.split(":")[1]!="undefined" else -1)

import matplotlib.pyplot as plt
def get_ratings(mturk, conditions, includ_particip=False):
    ratings = []
    std = []
    j=0
    second_dim = mturk.dim.to_list()
    first_dim = ["attractive" if m=="trustworthy" else "trustworthy" for m in second_dim]
    for i,(r,first,second) in enumerate(zip(mturk["ratings_list"].to_list(),first_dim,second_dim)):
      order = []
      for x in r.split(","):
        x = x.split(":")[0].replace(".png","").replace("level",first if len(order)<50 else second)
        if x not in order and x:
          order+=[x]
      
      if len(order)>100: print(len(order))
      rating = [mturk.loc[:,c].iloc[i] for c in [m for m in mturk.columns if "_Q" in m]]
    
      r = [k(filename)+(str(k(filename)),float(impression)) for filename,impression in zip(order,rating)]
      if np.std([float(rr[-1]) for rr in r])<8 or len(order)<100:
        continue
      std+=[np.std([float(rr[-1]) for rr in r])]
      # print(len(set(rating)))
      if all([c(r, rating) for c in conditions]):
        ratings += [m[:-2]+(m[-2].replace("('",f"('{j}', '"),m[-1]) for m in r] if includ_particip else r
        j+=1
    return ratings

ratings = get_ratings(mturk, [lambda r, rating :len(set(rating))>40])
print("Unique answers: ",a - len(ratings)/100)
a = len(ratings)/100
mturk2_unfiltered = pd.DataFrame(ratings).iloc[:,-2:]
# display(mturk2_unfiltered)
sd_unfiltered = mturk2_unfiltered.groupby(4).std()
means_unfiltered = mturk2_unfiltered.groupby(4).mean()
means_unfiltered

Unique answers:  34.0


,5
4,
"(""'0"", 'vector', 'attractive', '0')",30.538500
"(""'0"", 'vector', 'trustworthy', '0')",30.809500
"('0', 'flow', 'attractive', '0')",26.172750
"('0', 'flow', 'attractive', '1')",23.389714
"('0', 'flow', 'attractive', '2')",28.901622
...,...
"('9', 'vector', 'trustworthy', '0')",51.513077
"('9', 'vector', 'trustworthy', '1')",48.533333
"('9', 'vector', 'trustworthy', '2')",47.416897


In [13]:

def deviation(row):
    return np.mean([(abs(means_unfiltered.loc[r[-2]]-r[-1])/sd_unfiltered.loc[r[-2]])**2 for r in row])
ratings_filtered = get_ratings(mturk, [lambda r, rating :len(set(rating))>40, lambda r, rating:deviation(r)<2.],False)
print("Deviation: ",a - len(ratings_filtered)/100)
a = len(ratings_filtered)/100

mturk2 = pd.DataFrame(ratings_filtered).iloc[:,-2:]
# sd = mturk2.groupby(4).std()
means = mturk2.groupby(4).mean().reset_index()


Deviation:  24.0


In [14]:
import pandas as pd
import ast

# Assuming the data is in a DataFrame called 'df'
# If it's not, you'll need to load it first
df_b = means.copy()
df_b.columns = ['features', 'rating']
# df_b = df_b.loc[["trust" in m for m in df_b.features]]
# df_b["features"] = ["'".join(m.split("'")[:3])+"'"+"'".join(m.split("'")[5:]) for m in df_b["features"]]
# print(df_b.loc[["our" in m and "4" in m for m in df_b.features]].hist("rating"))
# df_b["features"] = [(m).replace('''"'0"''', "'0'").replace('''"4'"''', "'4'") for m in df_b.features]
df_b = df_b.loc[[not "'\"" in m and not "\"'" in m for m in df_b.features],:] 
# Step 1: Split the 'features' column into separate columns
df_b[['image', 'manip', 'dim', 'level']] = df_b['features'].apply(ast.literal_eval).tolist()

# # Step 2: Pivot the DataFrame to create columns for each 'value'
# df_b_pivot = df_b.pivot(index=['image', 'manip'], columns='value', values='rating')

# # Step 3: Reset the index to make 'image' and 'attribute' regular columns
# df_b_pivot.reset_index(inplace=True)

# # Step 4: Rename the columns for clarity
# df_b_pivot.columns.name = None
# # df_b_pivot.rename(columns={"-1": 'negative', "0.00": 'neutral', "1": 'positive'}, inplace=True)

# # Step 5: Sort the DataFrame by 'image' and 'attribute' for better readability
# df_b_pivot.sort_values(['image', 'manip'], inplace=True)
# for i in df_b_pivot.image.unique():
#     df_b_pivot.loc[(df_b_pivot.image == i)&(df_b_pivot.manip=="peterson.jpg"),'0'] = df_b_pivot.loc[(df_b_pivot.image == i)&(df_b_pivot.manip=="our.jpg"),'0'].item()
# # Display the result
# for r in df_b_pivot.manip.unique():
#     print(r)
#     (df_b_pivot[df_b_pivot.manip == r].drop(["image","manip"], axis=1).mean()).plot()
#     display(df_b_pivot[df_b_pivot.manip == r].drop(["image","manip"], axis=1).mean())


In [15]:
len(ratings_filtered)/100/10

29.3

In [17]:
df_c = df_b
df_c.drop(["features","image"], axis=1).groupby(["manip","dim","level"]).mean()

rating
manip  dim         level           
flow   attractive  0      54.151245
                   1      51.228176
                   2      55.666275
                   3      60.249188
                   4      62.369441
       trustworthy 0      51.490679
                   1      53.529611
                   2      55.531121
                   3      57.198715
                   4      58.055074
vector attractive  0      46.891230
                   1      48.595551
                   2      49.828260
                   3      50.515076
                   4      49.627377
       trustworthy 0      48.924245
                   1      51.147713
                   2      50.634778
                   3      51.707027
                   4      49.398096